In [ ]:
!pip install flask
!pip install flask_ngrok
!pip install wtforms
!pip install sqlalchemy
!pip install mysql-connector-python
!pip install transformers
!pip install sentencepiece

import tensorflow as tf
import os
import sys
sys.path.append('/content/gdrive/My Drive/ML/HanBert-54kN/')

import tokenization
from keras.models import load_model
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np
import pandas as pd
import tensorflow_hub as hub
from transformers import *

tokenizer = tokenization.FullTokenizer("/content/gdrive/My Drive/ML/HanBert-54kN/vocab_54k.txt", use_moran=True)

cls_token_id = 3
sep_token_id = 4
MAX_SEQUENCE_LENGTH = 140

def convert_to_inputs(text, tokenizer,max_seq_len):
    def return_id(str1,length):
        token = tokenizer.tokenize(text)
        if len(token) > max_seq_len-2:
            token = token[:max_seq_len-2]

        inputs_ids = tokenizer.convert_tokens_to_ids(token)    
        inputs_ids = [cls_token_id] + inputs_ids + [sep_token_id]
        input_mask = [1] * len(inputs_ids) #attention_mask

        padding_length = length - len(inputs_ids)
        inputs_ids = inputs_ids + ([0] * padding_length)
        input_mask = input_mask + ([0] * padding_length)
        segment_ids = [0] * len(inputs_ids) #token_type_ids
        return [inputs_ids, input_mask, segment_ids]

    input_ids, input_masks, input_segments = return_id(text, max_seq_len)

    return [input_ids, input_masks, input_segments]

def compute_outputs(df):
    return np.asarray(df['label'])
    
def compute_inputs(df, tokenizer, max_sequence_length):
    input_ids, segment_ids, input_mask = [], [], []
    for document in df['document']:
        inp_ids, inp_mask, seg_ids = convert_to_inputs(document, tokenizer, max_sequence_length)

        input_ids.append(inp_ids)
        segment_ids.append(seg_ids)
        input_mask.append(inp_mask)
    return [np.asarray(input_ids, dtype=np.int32),
            np.asarray(input_mask, dtype=np.int32), 
            np.asarray(segment_ids, dtype=np.int32)]

def create_model():
    ids = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    seg = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    
    config = BertConfig.from_json_file('/content/gdrive/My Drive/ML/HanBert-54kN/bert_config.json')
    config.output_hidden_states = False
    bert_model = TFBertModel.from_pretrained('/content/gdrive/My Drive/HanBert-54kN-torch.tar/HanBert-54kN-torch', config=config, from_pt=True)

    x = bert_model([ids, mask, seg])[0]

    
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.models.Model(inputs=[ids, mask, seg,], outputs=x)
    
    return model

def customLoadModel(model):
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])
    model.load_weights('/content/gdrive/My Drive/ML/HanBert-54kN/bert_model_nsmc_011.h5')

    return model

In [ ]:
import flask
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template, jsonify
from sklearn.externals import joblib
import numpy as np
from wtforms import Form, TextAreaField, validators

app = Flask(__name__)
run_with_ngrok(app)


@app.route("/predict", methods=['POST'])
def check():
    if request.method == 'POST':
        comment = request.json.get('comment')
        test = pd.DataFrame({ 'document' : [comment]})
        test_inputs = compute_inputs(test, tokenizer, MAX_SEQUENCE_LENGTH)

        # 입력데이터 예측
        prediction = loaded_model.predict(test_inputs)
        prob = (prediction * 100) if prediction >= 0.5 else ((1 - prediction) * 100)

        # 예측 값을 1차원 배열로부터 확인 가능한 문자열로 변환
        label = 'positive' if prediction >= 0.5 else 'negative'
    return jsonify({'result': label})

if __name__ == '__main__':
    # 모델 로드
    loaded_model = customLoadModel(model)
    # Flask 서비스 스타트
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://51970cb53f3e.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Apr/2021 13:26:56] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2021 13:27:32] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2021 13:28:00] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2021 13:28:14] "POST /predict HTTP/1.1" 200 -
